<a href="https://colab.research.google.com/github/sebesc/data_mgt_for_analytics/blob/main/covid_data_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -i */cases.csv

!csvstack *2020.csv *2021.csv >  daily_reports_total.csv

!csvcut -n daily_reports_total.csv

!head -n 100000 daily_reports_total.csv | csvstat

%load_ext sql

!dropdb -U student week11

!createdb -U student week11

%sql postgresql://student@/week11

%%sql
DROP TABLE IF EXISTS cases;

CREATE TABLE cases (
    province_states VARCHAR(50) NULL,
    country_region VARCHAR(20) NULL,
    last_update TIMESTAMP NULL,
    latitude NUMERIC(50) NULL,
    longitude NUMERIC(50) NULL,
    confirmed NUMERIC(50) NULL,
    deaths NUMERIC(50) NULL,
    recovered NUMERIC(50) NULL,
    active NUMERIC(50) NULL,
    fips NUMERIC(50) NULL,
    incident_rate NUMERIC(50) NULL,
    people_tested NUMERIC(50) NULL,
    people_hospitalized NUMERIC(50) NULL,
    mortality_rate NUMERIC(50) NULL,
    uid NUMERIC(50) NULL,
    iso3 VARCHAR(20) NULL,
    testing_rate NUMERIC(50) NULL,
    hospitalization_rate NUMERIC(50) NULL
)

%%sql  
SELECT * FROM CASES LIMIT 10

%%sql
COPY cases FROM '/home/ubuntu/daily_reports_total.csv'
CSV
HEADER;

%%sql
SELECT * from CASES 
limit 5

!wget https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/us_data/time_series/vaccine_data_us_timeline.csv

!csvstack *vaccine_data_us_timeline.csv >  vaccine_us.csv

!csvcut -n vaccine_us.csv

!head -n 100 vaccine_us.csv | csvstat

%%sql
DROP TABLE IF EXISTS vaccines;

CREATE TABLE vaccines (
    province_states VARCHAR(50) NULL,
    date TIMESTAMP NULL,
    vaccine_type VARCHAR(50) NULL,
    fips NUMERIC(50) NULL,
    country_region VARCHAR(50) NULL,
    latitude NUMERIC(50) NULL,
    longitude NUMERIC(50) NULL,
    doses_alloc NUMERIC(50) NULL,
    doses_shipped VARCHAR(50) NULL,
    doses_admin NUMERIC(50) NULL,
    stage_one_doses NUMERIC(50) NULL,
    stage_two_doses NUMERIC(50) NULL,
    combined_key VARCHAR(50) NULL
)

%%sql
COPY vaccines FROM '/home/ubuntu/vaccine_us.csv'
CSV
HEADER;
%%sql
SELECT * FROM vaccines
order by 2 desc
limit 10

zrvila@gwu.edu
Data Capture & Setup
Cases
In [1]:
# get file paths from BlackBoard
 !wget --user= --password= https://blackboard.gwu.edu/bbcswebdav/pid-11690379-dt-content-rid-96749166_2/xid-96749166_2

--2021-11-28 16:02:35--  https://blackboard.gwu.edu/bbcswebdav/pid-11690379-dt-content-rid-96749166_2/xid-96749166_2
Resolving blackboard.gwu.edu (blackboard.gwu.edu)... 3.223.166.8, 18.211.138.54, 54.224.33.22, ...
Connecting to blackboard.gwu.edu (blackboard.gwu.edu)|3.223.166.8|:443... connected.
HTTP request sent, awaiting response... 302 
Location: /bbcswebdav/pid-11690379-dt-content-rid-96749166_2/courses/67912_67913_202103/cases.csv [following]
--2021-11-28 16:02:35--  https://blackboard.gwu.edu/bbcswebdav/pid-11690379-dt-content-rid-96749166_2/courses/67912_67913_202103/cases.csv
Reusing existing connection to blackboard.gwu.edu:443.
HTTP request sent, awaiting response... 401 
Authentication selected: BASIC realm="blackboard.gwu.edu"
Reusing existing connection to blackboard.gwu.edu:443.
HTTP request sent, awaiting response... 200 
Length: 78336 (76K) [text/csv]
Saving to: ‘xid-96749166_2.1’

xid-96749166_2.1    100%[===================>]  76.50K  --.-KB/s    in 0.001s  

2021-11-28 16:02:36 (77.3 MB/s) - ‘xid-96749166_2.1’ saved [78336/78336]


In [ ]:
# get csv files for daily cases
!wget -i xid-96749166_2

In [1]:
#check row count of 1 file
!xsv count 11-27-2021.csv

58

In [2]:
#combine files into one csv file
!xsv cat rows *.csv > cases.csv
!xsv count cases.csv

34528

In [1]:
# show first 10 rows
!head -10 cases.csv | csvlook --max-columns -5

In [5]:
# print column headers
!csvcut -n cases.csv

 1: Province_State
  2: Country_Region
  3: Last_Update
  4: Lat
  5: Long_
  6: Confirmed
  7: Deaths
  8: Recovered
  9: Active
 10: FIPS
 11: Incident_Rate
 12: Total_Test_Results
 13: People_Hospitalized
 14: Case_Fatality_Ratio
 15: UID
 16: ISO3
 17: Testing_Rate
 18: Hospitalization_Rate

Vaccine Distribution
In [6]:
#fect file from GitHub
!wget https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/us_data/time_series/vaccine_data_us_timeline.csv

--2021-11-28 17:28:48--  https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/us_data/time_series/vaccine_data_us_timeline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10382165 (9.9M) [text/plain]
Saving to: ‘vaccine_data_us_timeline.csv’

vaccine_data_us_tim 100%[===================>]   9.90M  --.-KB/s    in 0.04s   

2021-11-28 17:28:48 (248 MB/s) - ‘vaccine_data_us_timeline.csv’ saved [10382165/10382165]


In [7]:
#rename file
!mv vaccine_data_us_timeline.csv vaccines.csv

In [34]:
!head -10 vaccines.csv | csvlook --max-columns 10

| Province_State |       Date | Vaccine_Type | FIPS | Country_Region |     Lat |    Long_ | Doses_alloc | Doses_shipped | Doses_admin | ... |
| -------------- | ---------- | ------------ | ---- | -------------- | ------- | -------- | ----------- | ------------- | ----------- | --- |
| Alabama        | 2020-12-10 | All          | True | US             | 32.318… | -86.902… |             |               |             | ... |
| Alabama        | 2020-12-10 | Moderna      | True | US             | 32.318… | -86.902… |             |               |             | ... |
| Alabama        | 2020-12-10 | Pfizer       | True | US             | 32.318… | -86.902… |             |               |             | ... |
| Alabama        | 2020-12-11 | All          | True | US             | 32.318… | -86.902… |             |               |             | ... |
| Alabama        | 2020-12-11 | Moderna      | True | US             | 32.318… | -86.902… |             |               |             | ... |
| Alabama        | 2020-12-11 | Pfizer       | True | US             | 32.318… | -86.902… |             |               |             | ... |
| Alabama        | 2020-12-12 | All          | True | US             | 32.318… | -86.902… |             |               |             | ... |
| Alabama        | 2020-12-12 | Moderna      | True | US             | 32.318… | -86.902… |             |               |             | ... |
| Alabama        | 2020-12-12 | Pfizer       | True | US             | 32.318… | -86.902… |             |               |             | ... |

In [9]:
!csvcut -n vaccines.csv

 1: Province_State
  2: Date
  3: Vaccine_Type
  4: FIPS
  5: Country_Region
  6: Lat
  7: Long_
  8: Doses_alloc
  9: Doses_shipped
 10: Doses_admin
 11: Stage_One_Doses
 12: Stage_Two_Doses
 13: Combined_Key

People Vaccinated
In [13]:
!wget https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/us_data/time_series/people_vaccinated_us_timeline.csv

--2021-11-28 17:30:57--  https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/us_data/time_series/people_vaccinated_us_timeline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1590477 (1.5M) [text/plain]
Saving to: ‘people_vaccinated_us_timeline.csv’

people_vaccinated_u 100%[===================>]   1.52M  --.-KB/s    in 0.008s  

2021-11-28 17:30:57 (188 MB/s) - ‘people_vaccinated_us_timeline.csv’ saved [1590477/1590477]


In [14]:
!mv people_vaccinated_us_timeline.csv people_vax.csv

In [15]:
!head -10 people_vax.csv | csvlook

| FIPS | Province_State | Country_Region |       Date |     Lat |    Long_ | Combined_Key | People_Fully_Vaccinated | People_Partially_Vaccinated |
| ---- | -------------- | -------------- | ---------- | ------- | -------- | ------------ | ----------------------- | --------------------------- |
| True | Alabama        | US             | 2020-12-10 | 32.318… | -86.902… | Alabama, US  |                         |                             |
| True | Alabama        | US             | 2020-12-11 | 32.318… | -86.902… | Alabama, US  |                         |                             |
| True | Alabama        | US             | 2020-12-12 | 32.318… | -86.902… | Alabama, US  |                         |                             |
| True | Alabama        | US             | 2020-12-13 | 32.318… | -86.902… | Alabama, US  |                         |                             |
| True | Alabama        | US             | 2020-12-14 | 32.318… | -86.902… | Alabama, US  |                         |                             |
| True | Alabama        | US             | 2020-12-15 | 32.318… | -86.902… | Alabama, US  |                         |                             |
| True | Alabama        | US             | 2020-12-16 | 32.318… | -86.902… | Alabama, US  |                         |                             |
| True | Alabama        | US             | 2020-12-17 | 32.318… | -86.902… | Alabama, US  |                         |                             |
| True | Alabama        | US             | 2020-12-18 | 32.318… | -86.902… | Alabama, US  |                         |                             |

In [16]:
!csvcut -n people_vax.csv

 1: FIPS
  2: Province_State
  3: Country_Region
  4: Date
  5: Lat
  6: Long_
  7: Combined_Key
  8: People_Fully_Vaccinated
  9: People_Partially_Vaccinated

Testing
In [17]:
!wget https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/testing_data/time_series_covid19_US.csv > tests.csv

--2021-11-28 17:31:11--  https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/testing_data/time_series_covid19_US.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2276070 (2.2M) [text/plain]
Saving to: ‘time_series_covid19_US.csv’

time_series_covid19 100%[===================>]   2.17M  --.-KB/s    in 0.01s   

2021-11-28 17:31:11 (185 MB/s) - ‘time_series_covid19_US.csv’ saved [2276070/2276070]


In [18]:
!mv time_series_covid19_US.csv tests.csv

In [37]:
!head -10 tests.csv | csvlook --max-columns 8

|       date | state | cases_conf_probable | cases_confirmed | cases_probable | tests_viral_positive | tests_viral_negative | tests_viral_total | ... |
| ---------- | ----- | ------------------- | --------------- | -------------- | -------------------- | -------------------- | ----------------- | --- |
| 2020-03-06 | AK    |                     |                 |                |                      |                      |                 8 | ... |
| 2020-03-07 | AK    |                     |                 |                |                      |                      |                12 | ... |
| 2020-03-08 | AK    |                     |                 |                |                      |                      |                14 | ... |
| 2020-03-09 | AK    |                     |                 |                |                      |                      |                23 | ... |
| 2020-03-10 | AK    |                     |                 |                |                      |                      |                23 | ... |
| 2020-03-11 | AK    |                     |                 |                |                      |                      |                46 | ... |
| 2020-03-12 | AK    |                     |                 |                |                      |                      |                46 | ... |
| 2020-03-13 | AK    |                     |                 |                |                      |                      |                60 | ... |
| 2020-03-14 | AK    |                     |                 |                |                      |                      |               144 | ... |

In [20]:
!csvcut -n tests.csv

 1: date
  2: state
  3: cases_conf_probable
  4: cases_confirmed
  5: cases_probable
  6: tests_viral_positive
  7: tests_viral_negative
  8: tests_viral_total
  9: tests_antigen_positive
 10: tests_antigen_total
 11: people_viral_positive
 12: people_viral_total
 13: people_antigen_positive
 14: people_antigen_total
 15: encounters_viral_total
 16: tests_combined_total

Summary statistics of files
In [24]:
!xsv stats cases.csv | xsv table

field                 type     sum                 min                  max                  min_length  max_length  mean                stddev
Province_State        Unicode                      Alabama              Wyoming              4           24                              
Country_Region        Unicode                      US                   US                   2           2                               
Last_Update           Unicode                      2020-04-12 23:18:15  2021-11-28 04:32:14  0           19                              
Lat                   Float    1227511.7750001159  -14.271              61.3707              0           19          36.840089285714534  10.789391869359381
Long_                 Float    -2839084.388200226  -170.1322            145.6739             0           18          -85.2066142917165   49.30821183888382
Confirmed             Integer  13255764774         0                    5060666              1           7           383913.4839550483   645982.1664372231
Deaths                Integer  243528856           0                    74152                1           5           7053.08317886933    11347.378809684322
Recovered             Float    1297302528          0                    2470308              0           9           85789.08398359988   193323.92050160456
Active                Float    2016169051          -120720              3542626              0           9           105652.62542577182  278904.17144552636
FIPS                  Float    113284777           1                    99999                0           7           3282.7603523718312  17243.86611833171
Incident_Rate         Float    208988544.51053703  0                    21139.09366954395    0           18          6272.165201396693   5108.871907479668
Total_Test_Results    Float    183774182827        3                    109974133            0           11          5516590.605079065   10469072.397697456
People_Hospitalized   Float    31549641            2                    89995                0           7           6151.226554883981   13789.744827318695
Case_Fatality_Ratio   Float    75205.80895537339   0                    70.37037037037037    0           19          2.2140193404196236  1.7609076999702908
UID                   Float    2650567949838       16                   84099999             2           10          76765753.8762169    23571546.984602317
ISO3                  Unicode                      ASM                  VIR                  3           3                               
Testing_Rate          Float    3028642506.535708   5.3917075537822825   564708.914147302     0           18          90914.73318331323   85896.70426099625
Hospitalization_Rate  Float    62286.01855755384   1.4184397163120568   38.501189532117365   0           18          12.143891315569046  5.244884778130014

In [25]:
!xsv stats vaccines.csv | xsv table

field            type     sum                 min          max          min_length  max_length  mean                stddev
Province_State   Unicode                      Alabama      Wyoming      4           30                              
Date             Unicode                      2020-12-10   2021-11-27   10          10                              
Vaccine_Type     Unicode                      All          unknown      3           10                              
FIPS             Integer  3463404             1            78           0           2           32.541002705952955  18.91358710786582
Country_Region   Unicode                      US           US           2           2                               
Lat              Float    3922220.377397655   -14.271      61.3707      0           19          36.85189019655803   10.794644546129854
Long_            Float    -9063407.389001267  -170.132     145.6739     0           18          -85.15678920813303  49.45868069343314
Doses_alloc      Integer  30403565067         0            27513735     0           8           1848690.5671287808  4621444.835229229
Doses_shipped    Integer  213633073377        -186039      70403965     0           8           2280577.244483541   5068737.634635182
Doses_admin      Integer  177787373606        -271346      58715000     0           8           1822899.3500051082  4173331.0400429564
Stage_One_Doses  Integer  97668404622         -1538        30886780     0           8           1734754.349336612   3259726.190538437
Stage_Two_Doses  Integer  72637698392         -212680      23136370     0           8           1131269.7346477888  2069039.2610722226
Combined_Key     Unicode                      Alabama, US  Wyoming, US  8           34                              

In [26]:
!xsv stats people_vax.csv | xsv table

field                        type     sum                  min          max          min_length  max_length  mean                stddev
FIPS                         Integer  643166               1            78           0           2           32.53571428571433   18.904803136574873
Province_State               Unicode                       Alabama      Wyoming      4           30                              
Country_Region               Unicode                       US           US           2           2                               
Date                         Unicode                       2020-12-10   2021-11-27   10          10                              
Lat                          Float    728254.8850001334    -14.271      61.3707      0           19          36.840089285714214  10.78939186935943
Long_                        Float    -1684364.3511997326  -170.132     145.6739     0           18          -85.2066142857144   49.30821182854585
Combined_Key                 Unicode                       Alabama, US  Wyoming, US  8           34                              
People_Fully_Vaccinated      Integer  40308952178          0            25395253     0           8           2106776.364292056   3223491.052117118
People_Partially_Vaccinated  Integer  9751951562           1515         7310293      0           7           528532.4135277203   817520.7813118588

In [27]:
!xsv stats tests.csv | xsv table

field                    type     sum           min       max        min_length  max_length  mean                stddev
date                     Unicode                1/1/2021  9/9/2021   8           10                              
state                    Unicode                AK        WY         2           2                               
cases_conf_probable      Integer  13123311054   0         4780867    0           7           371438.99278254085  627880.1534048168
cases_confirmed          Integer  7933277177    0         4780867    0           7           612371.8392126594   782911.8752024556
cases_probable           Integer  834211235     0         727876     0           6           64397.96472132161   100318.8010197748
tests_viral_positive     Integer  6904346507    0         4358633    0           7           417181.0578247722   653434.0996106125
tests_viral_negative     Integer  36560222667   1         39157516   0           8           3708309.4296581987  5322095.803230032
tests_viral_total        Integer  141468989932  0         109974133  0           9           5229907.206358556   9870637.188745363
tests_antigen_positive   Integer  559045399     0         526726     0           6           92114.91168232013   98159.61013729416
tests_antigen_total      Integer  10371032275   0         9030070    0           7           1054180.959036393   1253220.7048658219
people_viral_positive    Integer  11804350618   0         4780867    0           7           366549.20562663086  610276.4839324056
people_viral_total       Integer  26243729324   1         13374909   0           8           1666480.1450342867  2305225.5219011875
people_antigen_positive  Integer  87536746      3         165498     0           6           61300.24229691869   42172.209325365715
people_antigen_total     Integer  825597713     3         1243848    0           7           394456.6235069285   298731.890535025
encounters_viral_total   Integer  57808788842   0         80315133   0           8           6044415.395441246   11035506.540260032
tests_combined_total     Integer  184776202631  0         109974133  0           9           5217014.021994431   10253657.402755233



#########################################################################
Daily Code Data (with one day lag-- note that ‘day’ field is new field created)
%%sql
ALTER TABLE cases
ADD COLUMN day char(10);

%%sql
UPDATE cases
SET day = TO_CHAR(last_update, 'YYYY-MM-DD');

%%sql
select province_states,day,confirmed,deaths,incident_rate,people_tested,testing_rate,row_number() 
over(
order by province_states,day) as cases_id
from cases
where
province_states not in ('Virgin Islands','American Somoa','Puerto Rico','Northern Mariana Islands','Grand Princess','Guam','Recovered','Diamond Princess','Recovered')
limit 10;

%%sql
with a as (
    SELECT province_states,day,confirmed,
LAG(confirmed,1) over (
    order by province_states,day) previous_case_count
from CASES 
where province_states not in ('Virgin Islands','American Somoa','Puerto Rico','Northern Mariana Islands','Grand Princess','Guam','Recovered','Diamond Princess')
),
new_daily_cases as (
    select a.province_states,a.day,a.confirmed,a.previous_case_count,a.confirmed-coalesce(a.previous_case_count,0) as new_daily_cases
from a),
b as (
    select province_states,day,deaths,
    lag(deaths,1) over (
        order by province_states,day) previous_death_count
    from CASES
    where province_states not in ('Virgin Islands','American Somoa','Puerto Rico','Northern Mariana Islands','Grand Princess','Guam','Recovered','Diamond Princess')
),
new_daily_deaths as (
select b.province_states,b.day,b.deaths,b.previous_death_count,b.deaths-coalesce(b.previous_death_count,0) as new_daily_deaths
from b),
c as (
 select province_states,day,people_tested,
    lag(people_tested,1) over (
        order by province_states,day) previous_people_tested
    from CASES
    where province_states not in ('Virgin Islands','American Somoa','Puerto Rico','Northern Mariana Islands','Grand Princess','Guam','Recovered','Diamond Princess')
), 
new_people_tested as (
select c.province_states,c.day,c.people_tested,c.previous_people_tested,c.people_tested-coalesce(c.previous_people_tested,0) as new_people_tested
from c)
select new_daily_cases.province_states,new_daily_cases.day,new_daily_cases.confirmed,new_daily_cases.new_daily_cases,new_daily_deaths.deaths,new_daily_deaths.new_daily_deaths,new_people_tested.people_tested,new_people_tested.new_people_tested
from new_daily_cases left join new_daily_deaths
on new_daily_cases.province_states = new_daily_deaths.province_states and new_daily_cases.day=new_daily_deaths.day
left join new_people_tested
on new_daily_cases.province_states = new_people_tested.province_states and new_daily_cases.day=new_people_tested.day
order by 1,2
limit 10;

%%sql
select province_states, case when province_states in ('Alabama','Arkansas','Florida','Georgia','Kentucky','Louisiana',
                                                      'South Carolina','North Carolina','Mississippi','Tennessee') then 'South-East'
when province_states in ('Maine','Vermont','New Hampshire','Massachusetts','Rhode Island','Connecticut') then 'North-east'
when province_states in ('Pennsylvania','New York','New Jersey','Delaware','Maryland','Virginia','West Virginia','District of Columbia') then 'Mid-East'
when province_states in ('Michigan','Ohio','Indiana','Wisconsin','Minnesota','Illinois','Iowa') then 'Midwest'
when province_states in ('Idaho','Montana','Missouri','North Dakota','South Dakota','Nebraska') then 'New-Frontier'
when province_states in ('Utah','Colorado','Wyoming','Kansas') then 'Mountain-Region'
when province_states in ('California','Washington','Oregon','Nevada') then 'West-Coast'
when province_states in ('Texas','Arizona','New Mexico','Oklahoma') then 'Deep-South'
when province_states in ('Alaska','Hawaii') then 'Other' end as Region
from CASES
group by 1,2 
order by 1 asc;

%%sql
with a as (
    SELECT province_states,day,confirmed,
LAG(confirmed,1) over (
    order by province_states,day) previous_case_count
from CASES 
where province_states not in ('Virgin Islands','American Somoa','Puerto Rico','Northern Mariana Islands','Grand Princess','Guam','Recovered','Diamond Princess')
),
new_daily_cases as (
    select a.province_states,a.day,a.confirmed,a.previous_case_count,a.confirmed-coalesce(a.previous_case_count,0) as new_daily_cases
from a),
b as (
    select province_states,day,deaths,
    lag(deaths,1) over (
        order by province_states,day) previous_death_count
    from CASES
    where province_states not in ('Virgin Islands','American Somoa','Puerto Rico','Northern Mariana Islands','Grand Princess','Guam','Recovered','Diamond Princess')
),
new_daily_deaths as (
select b.province_states,b.day,b.deaths,b.previous_death_count,b.deaths-coalesce(b.previous_death_count,0) as new_daily_deaths
from b),
c as (
 select province_states,day,people_tested,
    lag(people_tested,1) over (
        order by province_states,day) previous_people_tested
    from CASES
    where province_states not in ('Virgin Islands','American Somoa','Puerto Rico','Northern Mariana Islands','Grand Princess','Guam','Recovered','Diamond Princess')
), 
new_people_tested as (
select c.province_states,c.day,c.people_tested,c.previous_people_tested,c.people_tested-coalesce(c.previous_people_tested,0) as new_people_tested
from c),
agg as (
select new_daily_cases.province_states,new_daily_cases.day,new_daily_cases.confirmed,new_daily_cases.new_daily_cases,new_daily_deaths.deaths,new_daily_deaths.new_daily_deaths,new_people_tested.people_tested,new_people_tested.new_people_tested
from new_daily_cases left join new_daily_deaths
on new_daily_cases.province_states = new_daily_deaths.province_states and new_daily_cases.day=new_daily_deaths.day
left join new_people_tested
on new_daily_cases.province_states = new_people_tested.province_states and new_daily_cases.day=new_people_tested.day
order by 1,2),

region as (
select province_states, case when province_states in ('Alabama','Arkansas','Florida','Georgia','Kentucky','Louisiana',
                                                      'South Carolina','North Carolina','Mississippi','Tennessee') then 'South-East'
when province_states in ('Maine','Vermont','New Hampshire','Massachusetts','Rhode Island','Connecticut') then 'North-east'
when province_states in ('Pennsylvania','New York','New Jersey','Delaware','Maryland','Virginia','West Virginia','District of Columbia') then 'Mid-East'
when province_states in ('Michigan','Ohio','Indiana','Wisconsin','Minnesota','Illinois','Iowa') then 'Midwest'
when province_states in ('Idaho','Montana','Missouri','North Dakota','South Dakota','Nebraska') then 'New-Frontier'
when province_states in ('Utah','Colorado','Wyoming','Kansas') then 'Mountain-Region'
when province_states in ('California','Washington','Oregon','Nevada') then 'West-Coast'
when province_states in ('Texas','Arizona','New Mexico','Oklahoma') then 'Deep-South'
when province_states in ('Alaska','Hawaii') then 'Other' end as region
from CASES
group by 1,2 
order by 1 asc),
one as (
select agg.province_states,region.region,agg.day,agg.confirmed,agg.new_daily_cases,agg.deaths,agg.new_daily_deaths,agg.people_tested,agg.new_people_tested
from agg left join region on agg.province_states=region.province_states)

select one.region,sum(one.new_daily_cases) as new_daily_cases,sum(one.new_daily_deaths) as new_daily_deaths,sum(one.new_people_tested) as new_people_tested
from one where one.day not like '2020-04-12'
group by 1;


Power BI Visuals

Code:

#name images below as shown to print in notebook file
from IPython.display import Image 
pil_img = Image(filename='Case Fatality Location Florida.png')
display(pil_img)

from IPython.display import Image 
pil_img = Image(filename='Case Fatality Location Midwest.png')
display(pil_img)

from IPython.display import Image 
pil_img = Image(filename='Positivity Rates.png')
display(pil_img)

from IPython.display import Image 
pil_img = Image(filename='Vaccine Availability.png')
display(pil_img)

from IPython.display import Image 
pil_img = Image(filename='Vax vs Deaths.png')
display(pil_img)
